In [8]:
import tensorflow as tf 
import tensorflow_io as tfio
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_addons as tfa
import cv2
from skimage import io, color
from tensorflow.data import AUTOTUNE

import time

from IPython import display

%matplotlib inline

In [9]:
tf.__version__

'2.10.1'

In [10]:
# data path 

face_path = r"D:\data\cartoonset10k_rgb"

In [16]:
images = os.listdir(path=face_path)
images = [os.path.join(face_path, image) for image in images if image.endswith(".jpg")]
len(images)

10000

In [17]:
list_ds = tf.data.Dataset.list_files(images, shuffle=False)
list_ds = list_ds.shuffle(len(images),reshuffle_each_iteration=False)

In [18]:
for f in list_ds.take(5):
    print(f.numpy())

b'D:\\data\\cartoonset10k_rgb\\cs10242808737376459572.jpg'
b'D:\\data\\cartoonset10k_rgb\\cs11086165772390352284.jpg'
b'D:\\data\\cartoonset10k_rgb\\cs10710077001642136367.jpg'
b'D:\\data\\cartoonset10k_rgb\\cs11066205443711286184.jpg'
b'D:\\data\\cartoonset10k_rgb\\cs11256838790393560917.jpg'


In [19]:
image_count = len(images)

val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [ ]:
width = 128
height = 128

In [22]:
def decode_img(file_path):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.read_file(file_path)
  
  img1 = tf.io.decode_jpeg(img, channels=3)
  img1 = tf.image.resize(img1, [width, height])
  img1 = tf.cast(img1, tf.float32) / 255.
  
  img3 = tfio.experimental.color.rgb_to_lab(img1)
  img2 = tf.image.rgb_to_grayscale(img1)
  return img2, img3[:,:,1:]

def get_segmented_image(img):
  
  img = cv2.resize(img, (width, height))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  pixel_values = np.float32( img.reshape((-1, 3)) )
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  k = 3
  _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  labels = labels.flatten()

  centers = np.uint8(centers)
  centers
  segmented_image = centers[labels.flatten()]
  segmented_image = segmented_image.reshape(img.shape)
  # show the image
  return img, segmented_image

def decode_img_1(file_path):
  
  img = tf.io.read_file(file_path)
  
  img = tf.io.decode_jpeg(img)
  
  img = np.array(img)
  
  img , img2= get_segmented_image(img)
  
  
  # numpy array to tensorflow tensor
  img1 = tf.convert_to_tensor(img1) / 255.
  img2 = tf.convert_to_tensor(img2) / 255.
  
  return img1, img2
  


def process_path(file_path):
  # Load the raw data from the file as a string
  
  img2, img3 = decode_img_1(file_path)
  return img2, img3


train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

NotImplementedError: in user code:

    File "C:\Users\vivek\AppData\Local\Temp\ipykernel_6660\1949632710.py", line 53, in process_path  *
        img2, img3 = decode_img_1(file_path)
    File "C:\Users\vivek\AppData\Local\Temp\ipykernel_6660\272505101.py", line 37, in decode_img_1  *
        img = np.array(img)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (DecodeJpeg:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


In [ ]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(16)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [ ]:
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(4):
  ax = plt.subplot(3, 3, i + 1)
  img = image_batch[i].numpy() 
  print(img.max())
  # img *= 255
  # i = i.astype("uint8")
  plt.imshow(img, cmap='gray')
#   label = label_batch[i]
  plt.title("img")
  plt.axis("off")

In [ ]:
image_batch1, image_batch2 = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(4):
  ax = plt.subplot(3, 3, i + 1)
  imgL = color.rgb2lab(color.gray2rgb(image_batch1[i].numpy()[:,:,0]))[:,:,0]
  img = image_batch2[i].numpy()
  img = np.dstack((imgL, img))
  img = color.lab2rgb(img)
  print(img.max())
  # img *= 255
  # i = i.astype("uint8")
  plt.imshow(img)
#   label = label_batch[i]
  plt.title("img")
  plt.axis("off")

In [ ]:
image_batch[0].shape, image_batch2[0].shape

In [ ]:
# Build A DC GAN model 

In [ ]:
# Build encoder decoder model using keras functional API

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(input_shape=(128, 128, 3)))
    
    model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(2, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    return model

generator = make_generator_model()
generator.summary()


def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[128, 128, 2]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


discriminator = make_discriminator_model()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
# Discriminator loss

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# generator loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images,out_images):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(images, training=True)

      real_output = discriminator(out_images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
import time

def generate_and_save_images(model, epoch, img_1, img2):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(img_1, training=False)

  fig = plt.figure(figsize=(80, 80))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      imgL = color.rgb2lab( color.gray2rgb(img_1[i].numpy()[:,:,0]))[:,:,:1]
      # print(imgL.shape,predictions[i].shape)
      img =  np.dstack((imgL,predictions[i]))
      img = color.lab2rgb(img)
      plt.imshow(img) # * 127.5 + 127.5)
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
  

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for img1,img2 in dataset:
      # img1 = tf.cast(img1, tf.float32)
      # img2 = tf.cast(img2, tf.float32)
      train_step(img1,img2)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             img1,img2)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           img1,img2)

In [ ]:
train(train_ds, 20)

In [ ]:
generator.save("D:/models/cartoon_generator")
discriminator.save("D:/models/cartoon_discriminator")

In [ ]:
i1 ,i2 =next(iter(val_ds))

In [ ]:
i1.shape

In [ ]:
color.rgb2lab( color.gray2rgb(i1[i].numpy()[:,:,0]))[:,:,:1].shape

In [ ]:
generate_and_save_images(generator,1,i1,i2)

In [ ]:
output = generator(i1, training = False)
print(output.shape)

In [ ]:
# test with sample image 
face_path = "D:/data/faces/obama.jpg"

img1, img2 = get_segmented_image(face_path)

In [ ]:
img2 = tf.expand_dims(img2, axis = 0)

In [ ]:
out = generator(img2, training = False)
out.shape

In [ ]:
imgL = color.rgb2lab( color.gray2rgb(img[0].numpy()[:,:,0]))[:,:,:1]
# print(imgL.shape,predictions[i].shape)
imgout =  np.dstack((imgL,out[0]))
imgout = color.lab2rgb(imgout)

In [ ]:
plt.imshow(imgL)

In [ ]:


lab2color = color.lab2rgb(i2[0])

plt.imshow(lab2color)